In [ ]:
pip install -q transformers==4.23.1

In [ ]:
# Libraries
!pip install sklearn
!pip install simpletransformers
!pip install rouge
!pip install transformers
!pip install sentence-transformers
!pip install nltk
!pip install gensim
!pip install sematch
!pip install pytorch
!pip install torch
import torch
import torch.nn as nn
import torch.optim as optim
import pickle
import re
import logging
import csv
import pandas as pd 
import numpy as np
import gensim
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
import sklearn
from sklearn.model_selection import train_test_split
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs
from simpletransformers.t5 import T5Model, T5Args
from rouge import Rouge
from gensim.parsing.preprocessing import remove_stopwords
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer, util
from gensim.models import KeyedVectors
from gensim.models import word2vec
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile

nltk.download('punkt')
nltk.download('stopwords')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=eb1265d0a3e9ae6ad3e46d8d26354886553f80f91cb1a1942e98c95c21a1efd3
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built sklearn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df = pd.read_csv("Base Data from scopus.csv", index_col=[0], skiprows=[20424])

df

,Authors,Author(s) ID,Title,Year,Source title,Cited by,DOI,Link,Affiliations,Authors with affiliations,...,Author Keywords,Index Keywords,Publisher,ISSN,ISBN,CODEN,Language of Original Document,Document Type,Publication Stage,Source
0,"Lentz D.L., Dunning N.P., Scarborough V.L., Ma...",7006721828;6602205930;6508093278;56543068500;2...,"Forests, fields, and the edge of sustainabilit...",2014,Proceedings of the National Academy of Science...,69.0,10.1073/pnas.1408631111,https://www.scopus.com/inward/record.uri?eid=2...,"Department of Biological Sciences, University ...","Lentz, D.L., Department of Biological Sciences...",...,Irrigation; Neotropics; Paleoecology; Paleoeth...,drinking water; rain; agroforestry; Antiquity;...,National Academy of Sciences,00278424,NaN,PNASA,English,Article,Final,Scopus
1,"Lopatto D., Hauser C., Jones C.J., Paetkau D.,...",8258144800;56821087600;56325380000;56099902500...,A central support system can facilitate implem...,2014,CBE Life Sciences Education,41.0,10.1187/cbe.13-10-0200,https://www.scopus.com/inward/record.uri?eid=2...,"Department of Psychology, Grinnell College, Gr...","Lopatto, D., Department of Psychology, Grinnel...",...,NaN,curriculum; education; educational model; geno...,American Society for Cell Biology,19317913,NaN,NaN,English,Article,Final,Scopus
2,Holman N.,6603593075;,Like Mixing Oil and Water? The Take-Up of Sust...,2014,Journal of Planning Education and Research,11.0,10.1177/0739456X14549468,https://www.scopus.com/inward/record.uri?eid=2...,"London School of Economics, London, United Kin...","Holman, N., London School of Economics, London...",...,hard-to-reach places; interpretive planning; p...,NaN,SAGE Publications Inc.,0739456X,NaN,NaN,English,Article,Final,Scopus
3,"Hirsch P.E., Schillinger S., Weigt H., Burkhar...",55338678300;56455185700;23973738300;7004305774;,A hydro-economic model for water level fluctua...,2014,PLoS ONE,24.0,10.1371/journal.pone.0114889,https://www.scopus.com/inward/record.uri?eid=2...,Research Centre for Sustainable Energy and Wat...,"Hirsch, P.E., Research Centre for Sustainable ...",...,NaN,drinking water; Article; climate change; contr...,Public Library of Science,19326203,NaN,POLNC,English,Article,Final,Scopus
4,"Zelenika I., Pearce J.M.",55433794100;7402030722;,Innovation through collaboration: scaling up s...,2014,"Environment, Development and Sustainability",14.0,10.1007/s10668-014-9528-7,https://www.scopus.com/inward/record.uri?eid=2...,"School of Environmental Studies, Queen’s Unive...","Zelenika, I., School of Environmental Studies,...",...,Appropriate technology; Crowd-sourcing; Knowle...,innovation; knowledge based system; scale effe...,Kluwer Academic Publishers,1387585X,NaN,EDSNB,English,Article,Final,Scopus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21538,"Yakubu H., Noorhidawati A., Kiran K.",56521512400;57198777461;57193664859;,Sustainability of digital collections for Nige...,2022,Malaysian Journal of Library and Information S...,NaN,10.22452/mjlis.vol27no1.5,https://www.scopus.com/inward/record.uri?eid=2...,"Department of Library and Information Science,...","Yakubu, H., Department of Library and Informat...",...,Academic libraries; Digital collections; Insti...,NaN,Faculty of Computer Science and Information Te...,13946234,NaN,NaN,English,Article,Final,Scopus
21539,"Allen K.J., Reide F., Gouramanis C., Keenan B....",7202632210;57704182700;36172890400;57222602802...,"Coupled insights from the palaeoenvironmental,...",2022,Environmental Research Letters,NaN,10.1088/1748-9326/ac6967,https://www.scopus.com/inward/record.uri?eid=2...,"School of Geography, Planning and Spatial Scie...","Allen, K.J., School of Geography, Planning and...",...,adaptive resilience; archaeological records; h...,Ecology; Feedback; Planning; Sustainable devel...,Institute of Physics,17489318,NaN,NaN,English,Article,Final,Scopus
21540,"Farkas M., Matolay R.",57704909500;55991770600;,Decision support for corporate s

In [ ]:
#Removing any duplicates
df = df.drop_duplicates(subset='Title', keep="first")

In [ ]:
len(df)

20352

In [ ]:
# Only keeping the abstract and title columns for the data
df = df.drop(['Authors', 'Author(s) ID', 'Year', 'Source title', 'Cited by', 'DOI',
         'Link', 'Affiliations', 'Authors with affiliations', 'Index Keywords',
         'Publisher', 'ISSN', 'ISBN', 'CODEN', 'Language of Original Document', 'Document Type', 'Publication Stage', 'Source'], axis=1)

In [ ]:
df

,Title,Abstract,Author Keywords
0,"Forests, fields, and the edge of sustainabilit...",Tikal has long been viewed as one of the leadi...,Irrigation; Neotropics; Paleoecology; Paleoeth...
1,A central support system can facilitate implem...,"In their 2012 report, the President’s Council ...",NaN
2,Like Mixing Oil and Water? The Take-Up of Sust...,Following studies about the adoption of sustai...,hard-to-reach places; interpretive planning; p...
3,A hydro-economic model for water level fluctua...,Water level fluctuations in lakes lead to shor...,NaN
4,Innovation through collaboration: scaling up s...,The open collaborative philosophy employed in ...,Appropriate technology; Crowd-sourcing; Knowle...
...,...,...,...
21538,Sustainability of digital collections for Nige...,Collection management in libraries has evolved...,Academic libraries; Digital collections; Insti...
21539,"Coupled insights from the palaeoenvironmental,...",Many governments and organisations are current...,adaptive resilience; archaeological records; h...
21540,Decision support for corporate sustainability:...,Decision support systems for corporate environ...,employee; end-user; environmental decision sup...
21541,Sustainability reporting quality and the finan...,"Purpose: Based on institutional theory, this p...",China; CSR disclosures; Financial sector; Inst...


In [ ]:
df = df.dropna( how='any', subset=['Author Keywords'])

df = df.reset_index(drop=True)

df.to_csv("1D.csv", index=False)


In [ ]:
df = pd.read_csv("1D.csv")

In [ ]:
df

,Title,Abstract,Author Keywords
0,"Forests, fields, and the edge of sustainabilit...",Tikal has long been viewed as one of the leadi...,Irrigation; Neotropics; Paleoecology; Paleoeth...
1,Like Mixing Oil and Water? The Take-Up of Sust...,Following studies about the adoption of sustai...,hard-to-reach places; interpretive planning; p...
2,Innovation through collaboration: scaling up s...,The open collaborative philosophy employed in ...,Appropriate technology; Crowd-sourcing; Knowle...
3,Sustainability in the Malaysian palm oil industry,"With a rapidly growing world population, the d...",Agent-based modelling; Eco-labelling; Informat...
4,The moral muteness of managers: An Anglo-Ameri...,Several studies in the Anglo-American context ...,business ethics; cross-cultural; cross-nationa...
...,...,...,...
18741,Sustainability of digital collections for Nige...,Collection management in libraries has evolved...,Academic libraries; Digital collections; Insti...
18742,"Coupled insights from the palaeoenvironmental,...",Many governments and organisations are current...,adaptive resilience; archaeological records; h...
18743,Decision support for corporate sustainability:...,Decision support systems for corporate environ...,employee; end-user; environmental decision sup...
18744,Sustainability reporting quality and the finan...,"Purpose: Based on institutional theory, this p...",China; CSR disclosures; Financial sector; Inst...


In [ ]:
df = df[['Abstract','Title','Author Keywords']]

df.rename(columns = {'Title':'target_text', 'Abstract':'input_text', 'Author Keywords':'Keywords'}, inplace = True)

df

,input_text,target_text,Keywords
0,Tikal has long been viewed as one of the leadi...,"Forests, fields, and the edge of sustainabilit...",Irrigation; Neotropics; Paleoecology; Paleoeth...
1,Following studies about the adoption of sustai...,Like Mixing Oil and Water? The Take-Up of Sust...,hard-to-reach places; interpretive planning; p...
2,The open collaborative philosophy employed in ...,Innovation through collaboration: scaling up s...,Appropriate technology; Crowd-sourcing; Knowle...
3,"With a rapidly growing world population, the d...",Sustainability in the Malaysian palm oil industry,Agent-based modelling; Eco-labelling; Informat...
4,Several studies in the Anglo-American context ...,The moral muteness of managers: An Anglo-Ameri...,business ethics; cross-cultural; cross-nationa...
...,...,...,...
18741,Collection management in libraries has evolved...,Sustainability of digital collections for Nige...,Academic libraries; Digital collections; Insti...
18742,Many governments and organisations are current...,"Coupled insights from the palaeoenvironmental,...",adaptive resilience; archaeological records; h...
18743,Decision support systems for corporate environ...,Decision support for corporate sustainability:...,employee; end-user; environmental decision sup...
18744,"Purpose: Based on institutional theory, this p...",Sustainability reporting quality and the finan...,China; CSR disclosures; Financial sector; Inst...


In [ ]:
## I need keywords in a list format
df['Keywords'] = df['Keywords'].str.split(';')

In [ ]:
df['Keywords'][0]
## There is a space after the first word, consider before similarity measures.

['Irrigation',
 ' Neotropics',
 ' Paleoecology',
 ' Paleoethnobotany',
 ' Root crops']

In [ ]:
df['input_text'] = df['input_text'].str.split('©').str[0]

In [ ]:
df['input_text'] = df['input_text'].apply(lambda x: x.replace('\n',''))

In [ ]:
df['input_text'] = df['input_text'].apply(lambda x : re.sub('\s*\(.*?\)\s*', '', x))

In [ ]:
df.to_csv("Data for 1st run.csv", index=False)

READING DATA FOR SPLITTING AND TRAINING FROM HERE. I am leaving the keywords out for now. 

In [ ]:
df = pd.read_csv("Data for 1st run.csv")

df['target_text'] = df['target_text'].apply(lambda x : re.sub('\s*\(.*?\)\s*', '', x))

df['target_text'] = df['target_text'].apply(lambda x: x.replace('\n',''))

## Only keywords ## Removing whitespaces and lower()
df['Keywords'] = df['Keywords'].str.lower()
df['Keywords'] = df['Keywords'].str.replace("' ", "'")


In [ ]:
len(df)

18746

In [ ]:
df['Keywords'] = df['Keywords'].str.replace(',', '')
df['Keywords'] = df['Keywords'].str.replace('\'', '')
df['Keywords'] = df['Keywords'].str.replace('[', '')
df['Keywords'] = df['Keywords'].str.replace(']', '')

df

<ipython-input-7-b713d9d1c142>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Keywords'] = df['Keywords'].str.replace('[', '')
<ipython-input-7-b713d9d1c142>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Keywords'] = df['Keywords'].str.replace(']', '')


,input_text,target_text,Keywords
0,Tikal has long been viewed as one of the leadi...,"Forests, fields, and the edge of sustainabilit...",irrigation neotropics paleoecology paleoethnob...
1,Following studies about the adoption of sustai...,Like Mixing Oil and Water? The Take-Up of Sust...,hard-to-reach places interpretive planning pla...
2,The open collaborative philosophy employed in ...,Innovation through collaboration: scaling up s...,appropriate technology crowd-sourcing knowledg...
3,"With a rapidly growing world population, the d...",Sustainability in the Malaysian palm oil industry,agent-based modelling eco-labelling informatio...
4,Several studies in the Anglo-American context ...,The moral muteness of managers: An Anglo-Ameri...,business ethics cross-cultural cross-national ...
...,...,...,...
18741,Collection management in libraries has evolved...,Sustainability of digital collections for Nige...,academic libraries digital collections institu...
18742,Many governments and organisations are current...,"Coupled insights from the palaeoenvironmental,...",adaptive resilience archaeological records his...
18743,Decision support systems for corporate environ...,Decision support for corporate sustainability:...,employee end-user environmental decision suppo...
18744,"Purpose: Based on institutional theory, this p...",Sustainability reporting quality and the finan...,china csr disclosures financial sector institu...


ONCE AGAIN TO CLARIFY, USE "KEYWORDS_LIST" FOR W2VEC, FASTTEXT AND SEMATCH

USE "KEYWORDS" FOR TRANSFORMER BASED, BLEU AND ROUGE


In [ ]:
## Both BART and T5 need prefix for summarization task
# Leaving keyword column out for now
df["prefix"] = "summarize"

df = df[['prefix', 'input_text', 'target_text', 'Keywords']]

In [ ]:
df

,prefix,input_text,target_text,Keywords
0,summarize,Tikal has long been viewed as one of the leadi...,"Forests, fields, and the edge of sustainabilit...",irrigation neotropics paleoecology paleoethnob...
1,summarize,Following studies about the adoption of sustai...,Like Mixing Oil and Water? The Take-Up of Sust...,hard-to-reach places interpretive planning pla...
2,summarize,The open collaborative philosophy employed in ...,Innovation through collaboration: scaling up s...,appropriate technology crowd-sourcing knowledg...
3,summarize,"With a rapidly growing world population, the d...",Sustainability in the Malaysian palm oil industry,agent-based modelling eco-labelling informatio...
4,summarize,Several studies in the Anglo-American context ...,The moral muteness of managers: An Anglo-Ameri...,business ethics cross-cultural cross-national ...
...,...,...,...,...
18741,summarize,Collection management in libraries has evolved...,Sustainability of digital collections for Nige...,academic libraries digital collections institu...
18742,summarize,Many governments and organisations are current...,"Coupled insights from the palaeoenvironmental,...",adaptive resilience archaeological records his...
18743,summarize,Decision support systems for corporate environ...,Decision support for corporate sustainability:...,employee end-user environmental decision suppo...
18744,summarize,"Purpose: Based on institutional theory, this p...",Sustainability reporting quality and the finan...,china csr disclosures financial sector institu...


In [ ]:
## Dividing to train and test
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
## Dividing test to eval and pred data
eval_data, pred_data = train_test_split(test_data, test_size=0.02, random_state=42)

## From here on, train_data and eval_data for training
## pred_data for unseen prediction

In [ ]:
pred_data['target_text']

10608    Food security and sustainability: Exploring Ja...
14040    Canada's oil sands industry from a sustainabil...
46       Perspectives on sustainability in humanitarian...
17518    Students’ Learning on Sustainable Development ...
18577    INNOVATION: A MODERATING FACTOR FOR THE EFFECT...
                               ...                        
5572     Putting resilient sustainability into strategy...
2458     A comparison of electricity generation system ...
7341     Taxi downsizing: A new approach to efficiency ...
15889    Analysis of OpenStreetMap Data Quality for Sel...
8962     Analysis of scale factors on China's sustainab...
Name: target_text, Length: 75, dtype: object

In [ ]:
pred_data['target_text'][14081]

'Becoming child and sustainability—the kindergarten teacher as agency mobiliser for sustainability through keeping the concept of the child in play'

In [ ]:
pred_data.to_csv("Pred_data.csv")

In [ ]:
pred_data.to_csv("Prediction data.csv")

In [ ]:
pred_data

keyword_col = pred_data['Keywords']
train_data = train_data.drop('Keywords', axis=1)
eval_data = eval_data.drop('Keywords', axis=1)
pred_data = pred_data.drop('Keywords', axis=1)

In [ ]:
train_data

,prefix,input_text,target_text
6069,summarize,Sustainability of water supply services has be...,Water price optimization after the Melamchi Wa...
6311,summarize,The United NationsAgenda 2030 represents the s...,Relationship between the sustainable developme...
5090,summarize,The importance of early detection of autism sp...,Sustainability of an early detection program f...
17796,summarize,The use of alternative sources to produce less...,Use of mild reaction conditions to improve qua...
10794,summarize,Policies on sustainability of buildings have b...,An overview of sustainability policies and str...
...,...,...,...
11284,summarize,Small and medium-sized enterprisesare recogniz...,Risk management implementation for sustainable...
11964,summarize,"The unique strengths of qualitative research, ...",A path forward for qualitative research on sus...
5390,summarize,The purpose of this study is to assess the com...,Occupational health and safety disclosures in ...
860,summarize,"Background: A so called "" taxonomic impediment...",Integrative taxonomy on the fast track - towar...


In [ ]:
eval_data

,prefix,input_text,target_text
14699,summarize,Social sustainability is a major part of both ...,Exploring social sustainability handprint—part...
2635,summarize,This paper focuses on the sustainability analy...,Multi-criteria decision making in the sustaina...
17377,summarize,"In several European countries, a new football ...",Striving for Social Sustainability of Football...
852,summarize,India is a poster child for groundwater deplet...,Assessing chronic and climate-induced water ri...
17037,summarize,This study employs a holistic and indigenous t...,The sustainable development of Asian students’...
...,...,...,...
5438,summarize,The Accelerating Children's HIV/AIDS Treatment...,Sustainability and Accelerating Children's HIV...
16483,summarize,This study analyzes the influence of corporate...,Sustainable Development and Customer Satisfact...
13427,summarize,A sustainability indexis a benchmark for guidi...,A sustainability index within water and wastew...
6730,summarize,Sustainable developmenthas become a fundamenta...,The Evolution of Sustainable Development Theor...


In [ ]:
pred_data

,prefix,input_text,target_text
10608,summarize,There are many challenges in assuring the sust...,Food security and sustainability: Exploring Ja...
14040,summarize,The present study aims to investigate the impa...,Canada's oil sands industry from a sustainabil...
46,summarize,Purpose– The purpose of this paper is to devel...,Perspectives on sustainability in humanitarian...
17518,summarize,Sustainable social-resource-based community ma...,Students’ Learning on Sustainable Development ...
18577,summarize,The study aims to examine the effect of innova...,INNOVATION: A MODERATING FACTOR FOR THE EFFECT...
...,...,...,...
5572,summarize,Purpose: The purpose of this paper is to repor...,Putting resilient sustainability into strategy...
2458,summarize,Planning for electricity generation systems is...,A comparison of electricity generation system ...
7341,summarize,Taxis are considered one of the symbols of urb...,Taxi downsizing: A new approach to efficiency ...
15889,summarize,One potential source of geospatial open data f...,Analysis of OpenStreetMap Data Quality for Sel...


In [ ]:
## For BLEU, Rouge, and Transformer based similarity, this format is ok.
## For W2vec, Fasttext and sematch, this needs to be converted into a list of lists. 
keywords = keyword_col.tolist()

In [ ]:
keywords

['ageing farmers food security japan agriculture sustainable agriculture youth',
 'energy environment sustainable development',
 'aid effectiveness contingency theory disaster relief humanitarian organisation humanitarian supply chain sustainability',
 'educational intervention grounded theory approach japan rural sustainable development goal university',
 'concentric diversification conglomerate diversification diversification strategies hospitality firms innovation sustainability',
 'conceptual framework material selection preliminary design phase sustainability',
 'efqm excellence model higher education institutions organizational excellence quality quality management sustainable development',
 'intermediaries port sector transition work',
 'covid-19 editorial events festivals social sustainability',
 'crisis distributed manufacturing global south humanitarian open hardware sustainable development',
 'cooling and lubrication systems intermittent facing intermittent turning magnesium

In [ ]:
## This code will convert the keywords to a list of lists
## Only for w2vec, fasttext and sematch
keywords_list = []

for i in range(len(keywords)):
  keywords_list.append(keywords[i].split(" "))

In [ ]:
keywords_list

[['ageing',
  'farmers',
  'food',
  'security',
  'japan',
  'agriculture',
  'sustainable',
  'agriculture',
  'youth'],
 ['energy', 'environment', 'sustainable', 'development'],
 ['aid',
  'effectiveness',
  'contingency',
  'theory',
  'disaster',
  'relief',
  'humanitarian',
  'organisation',
  'humanitarian',
  'supply',
  'chain',
  'sustainability'],
 ['educational',
  'intervention',
  'grounded',
  'theory',
  'approach',
  'japan',
  'rural',
  'sustainable',
  'development',
  'goal',
  'university'],
 ['concentric',
  'diversification',
  'conglomerate',
  'diversification',
  'diversification',
  'strategies',
  'hospitality',
  'firms',
  'innovation',
  'sustainability'],
 ['conceptual',
  'framework',
  'material',
  'selection',
  'preliminary',
  'design',
  'phase',
  'sustainability'],
 ['efqm',
  'excellence',
  'model',
  'higher',
  'education',
  'institutions',
  'organizational',
  'excellence',
  'quality',
  'quality',
  'management',
  'sustainable',
  'd

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
## BART Model
## Model Arguments

model_args = Seq2SeqArgs()
model_args.do_sample = True
model_args.eval_batch_size = 16
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 2500
model_args.evaluate_during_training_verbose = True
model_args.fp16 = False
model_args.learning_rate = 3e-5
model_args.max_length = 128
model_args.max_seq_length = 128
model_args.num_return_sequences = 3
model_args.num_train_epochs = 3
model_args.optimizer = "AdamW"
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_steps = -1
model_args.top_k = 50
model_args.top_p = 0.95
model_args.train_batch_size = 8
model_args.use_multiprocessing = False
model_args.wandb_project = "Summarization with BART"

In [ ]:
# Model intialization
model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-large",
    args=model_args,
)


In [ ]:
## Training here
model.train_model(train_data, eval_data=eval_data)

  0%|          | 0/14996 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


...

  0%|          | 0/3675 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/1875 [00:00<?, ?it/s]

  0%|          | 0/3675 [00:00<?, ?it/s]

  0%|          | 0/3675 [00:00<?, ?it/s]

(5625,
 {'global_step': [1875, 2500, 3750, 5000, 5625],
  'eval_loss': [0.4019435979749845,
   0.3951545992623205,
   0.3849694629078326,
   0.38443669443545136,
   0.3825321999580964],
  'train_loss': [0.4871711730957031,
   0.6108761429786682,
   0.27908414602279663,
   0.4321862459182739,
   0.31595128774642944]})

In [ ]:
predo_data = pd.read_csv("new data scopus.csv")

In [ ]:
len(predo_data)

1195

In [ ]:
predo_data = predo_data[predo_data['Abstract'] != '[No abstract available]']

In [ ]:
len(predo_data)

1126

In [ ]:
predo_data = predo_data.head(500)

In [ ]:
predo_data

,Title,Abstract
0,Energy efficiency and sustainable development ...,This study attempts to connect Sustainable Dev...
1,Assessment of the pre-combustion carbon captur...,The rapid progress in the global population an...
2,"Global, regional, and national causes of under...",Background: Causes of mortality are a crucial ...
3,Agricultural land systems importance for suppo...,Agriculture provides the largest share of food...
4,Global guidelines and requirements for profess...,The future limitation of certain types of mine...
...,...,...
1190,Achieving the Sustainable Development Goals Th...,Education plays a decisive role in achieving t...
1191,Creative sustainable development goal-based ph...,This article assesses the effects on creativit...
1192,"Sustainable Development Goals, Conflict, and F...",Good governance is critical in creating a cond...
1193,Environmental Safety in the Sustainable Develo...,This chapter looks at three aspects of environ...


In [ ]:
predo_data = predo_data[['Abstract','Title']]

predo_data.rename(columns = {'Title':'target_text', 'Abstract':'input_text'}, inplace = True)

predo_data['input_text'] = predo_data['input_text'].str.split('©').str[0]
predo_data['input_text'] = predo_data['input_text'].apply(lambda x: x.replace('\n',''))
predo_data['input_text'] = predo_data['input_text'].apply(lambda x : re.sub('\s*\(.*?\)\s*', '', x))


In [ ]:
predo_data

,input_text,target_text
0,This study attempts to connect Sustainable Dev...,Energy efficiency and sustainable development ...
1,The rapid progress in the global population an...,Assessment of the pre-combustion carbon captur...
2,Background: Causes of mortality are a crucial ...,"Global, regional, and national causes of under..."
3,Agriculture provides the largest share of food...,Agricultural land systems importance for suppo...
4,The future limitation of certain types of mine...,Global guidelines and requirements for profess...
...,...,...
1190,Education plays a decisive role in achieving t...,Achieving the Sustainable Development Goals Th...
1191,This article assesses the effects on creativit...,Creative sustainable development goal-based ph...
1192,Good governance is critical in creating a cond...,"Sustainable Development Goals, Conflict, and F..."
1193,This chapter looks at three aspects of environ...,Environmental Safety in the Sustainable Develo...


In [ ]:
predo_data = predo_data.reset_index(drop=True)

predo_data

,input_text,target_text
0,This study attempts to connect Sustainable Dev...,Energy efficiency and sustainable development ...
1,The rapid progress in the global population an...,Assessment of the pre-combustion carbon captur...
2,Background: Causes of mortality are a crucial ...,"Global, regional, and national causes of under..."
3,Agriculture provides the largest share of food...,Agricultural land systems importance for suppo...
4,The future limitation of certain types of mine...,Global guidelines and requirements for profess...
...,...,...
1121,Education plays a decisive role in achieving t...,Achieving the Sustainable Development Goals Th...
1122,This article assesses the effects on creativit...,Creative sustainable development goal-based ph...
1123,Good governance is critical in creating a cond...,"Sustainable Development Goals, Conflict, and F..."
1124,This chapter looks at three aspects of environ...,Environmental Safety in the Sustainable Develo...


In [ ]:
predo_data["prefix"] = "summarize"

predo_data = predo_data[['prefix', 'input_text', 'target_text']]

In [ ]:
predo_data

,prefix,input_text,target_text
0,summarize,This study attempts to connect Sustainable Dev...,Energy efficiency and sustainable development ...
1,summarize,The rapid progress in the global population an...,Assessment of the pre-combustion carbon captur...
2,summarize,Background: Causes of mortality are a crucial ...,"Global, regional, and national causes of under..."
3,summarize,Agriculture provides the largest share of food...,Agricultural land systems importance for suppo...
4,summarize,The future limitation of certain types of mine...,Global guidelines and requirements for profess...
...,...,...,...
1121,summarize,Education plays a decisive role in achieving t...,Achieving the Sustainable Development Goals Th...
1122,summarize,This article assesses the effects on creativit...,Creative sustainable development goal-based ph...
1123,summarize,Good governance is critical in creating a cond...,"Sustainable Development Goals, Conflict, and F..."
1124,summarize,This chapter looks at three aspects of environ...,Environmental Safety in the Sustainable Develo...


In [ ]:
pred_data['prefix'] = df['prefix'].astype(object)
pred_data['input_text'] = df['input_text'].astype(object)
pred_data['target_text'] = df['target_text'].astype(object)

NameError: ignored

In [ ]:
predo_data['prefix']

0       summarize
1       summarize
2       summarize
3       summarize
4       summarize
          ...    
1121    summarize
1122    summarize
1123    summarize
1124    summarize
1125    summarize
Name: prefix, Length: 1126, dtype: object

In [ ]:
predo_data['input_text']

0       This study attempts to connect Sustainable Dev...
1       The rapid progress in the global population an...
2       Background: Causes of mortality are a crucial ...
3       Agriculture provides the largest share of food...
4       The future limitation of certain types of mine...
                              ...                        
1121    Education plays a decisive role in achieving t...
1122    This article assesses the effects on creativit...
1123    Good governance is critical in creating a cond...
1124    This chapter looks at three aspects of environ...
1125    The United Nations Sustainable Development Goa...
Name: input_text, Length: 1126, dtype: object

In [ ]:
## Loop to go through the 500 pred data
to_predict = [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(predo_data["prefix"].tolist(), predo_data["input_text"].tolist())
]

# truth = pred_data["target_text"].tolist()

In [ ]:
## Making predictions on the 75 pred records
# preds = model.predict(to_predict)

# preds = model.predict(to_predict)
new_preds = model.predict(to_predict)

# Preds has the prediction results ## Truth has the original title text

Generating outputs:   0%|          | 0/71 [00:00<?, ?it/s]

In [ ]:
new_preds

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
preds

[['The link between sustainable development and energy efficiency in the Asian and Pacific: evidence from a data envelopment analysis of 20 countries',
  'Sustainable development and energy efficiency: The role of green innovation and financial development in emerging economies',
  'Connecting sustainable development goals with energy efficiency: A global analysis'],
 ['Climate change mitigation: An essential aspect to meeting the global goal of sustainable development',
  'The impact of climate change mitigation efforts on economic growth and the environment in the light of the Paris climate agreement',
  'The importance of energy efficiency for sustainable development and economic growth'],
 ['Implementation of cause-specific mortality estimates for children in Ghana from 2000 to 2019: implications for child survival and sustainability and the need for new health systems',
  'New cause-specific mortality estimates to assess and improve sustainability',
  'Revisiting cause-specific mo

In [ ]:
## T5 Model
## Model arguments

model_args = T5Args()
model_args.do_sample = True
model_args.eval_batch_size = 16
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 2500
model_args.evaluate_during_training_verbose = True
model_args.fp16 = False
model_args.learning_rate = 3e-5
model_args.max_length = 128
model_args.max_seq_length = 128
model_args.num_beams = None
model_args.num_return_sequences = 3
model_args.num_train_epochs = 3
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_steps = -1
model_args.top_k = 50
model_args.top_p = 0.95
model_args.train_batch_size = 8
model_args.use_multiprocessing = False
model_args.wandb_project = "Summarization with T5"

In [ ]:
# Model intialization
model = T5Model("t5", "t5-base", args=model_args)

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:
# Model training
model.train_model(train_data, eval_data=eval_data)

  0%|          | 0/14996 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3672: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3547: UserWarning: `as_target_tokenizer` is deprec

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: safwatn. Use `wandb login --relogin` to force relogin


Running Epoch 0 of 3:   0%|          | 0/1875 [00:00<?, ?it/s]

...

  0%|          | 0/3675 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3672: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3672: FutureWarning: 
`prepare_seq2seq_batch` is d

  0%|          | 0/3675 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3672: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3547: UserWarning: `as_target_tokenizer` is deprec

(5625,
 {'global_step': [1875, 2500, 3750, 5000, 5625],
  'eval_loss': [2.270338535308838,
   2.2463080084842186,
   2.208660632631053,
   2.1941611683886983,
   2.1926914049231487],
  'train_loss': [2.794530153274536,
   2.418307065963745,
   2.7383291721343994,
   2.4713189601898193,
   2.5328948497772217]})

In [ ]:
## Loop to go through the 75 pred data
to_predict = [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(pred_data["prefix"].tolist(), pred_data["input_text"].tolist())
]

truth = pred_data["target_text"].tolist()

In [ ]:
## Making predictions on the 75 pred records
preds = model.predict(to_predict)

# Preds has the prediction results ## Truth has the original title text

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

TypeError: ignored

In [ ]:
preds

[['Youth and food sustainability: A qualitative study in Japan',
  'Youth involvement in food sustainability in Japan: A phenomenological study',
  'The youth and agriculture in Japan: A phenomenological approach for the sustainability of food supply'],
 ['The impact of oil sands development on economic sustainability',
  'Three pillars of sustainability: A study of the oil sands industry in Canada',
  'Sustainability of oil sands development in Canada'],
 ['Insights into the contextual context and expectations of humanitarian organisations on sustainability',
  'Sustainable development based on societal expectations of humanitarian organisations',
  'Managing social expectations and sustainability: The case of humanitarian organisations'],
 ['A study of student learning theory and perspectives on sustainable development goals in rural Japan',
  'Education for sustainability goals in rural Japanese universities: Study on the learning and attitude of rural university students',
  'The E